In [ ]:
from tsai.all import *
import pandas as pd
from datenverarbeitung.dataloader import dataloader
CUDA_LAUNCH_BLOCKING=1

num_scenario = 1
dl = dataloader(scenario= num_scenario, path="/home/adi/cloudy_adlu/smart_hans/AP2/Daten/single_file_for_testing", nr_taps=1, move_window_by=-10, feature_list=["pitch"])
train, test= dl.get_train_test(frac = 0.8, seed = 0)

X_test, y_test = df2xy(test, target_col='target')
X_train, y_train = df2xy(train, target_col='target')

X, y, splits = combine_split_data([X_train, X_test], [y_train, y_test])

# X_train, y_train = df2xy(train_df, target_col='target')
# np.shape(X)
# test_eq(X_test.shape, (60, 1, 40))
# test_eq(y_test.shape, (60, ))

tfms  = [None, [Categorize()]]
dsets = TSDatasets(X, y, tfms=tfms, splits=splits, inplace=True)
dsets

dls = TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=[64, 128], batch_tfms=[TSStandardize()], num_workers=0)

dls.show_batch(sharey=True)

: 

In [ ]:
scenario_name_stage0 = "scenario_{}_stage0".format(num_scenario)
model = InceptionTime(dls.vars, dls.c)
learn = ts_learner(dls, model, metrics=accuracy, train_metrics=True)
learn.save(scenario_name_stage0)

In [ ]:
learn.load(scenario_name_stage0)
learn.lr_find()

In [ ]:
scenario_name_stage1 = scenario_name_stage0.replace("0","1")
learn.fit_one_cycle(100, lr_max=1e-3)
learn.save(scenario_name_stage1)

In [ ]:
learn.recorder.plot_metrics()

In [ ]:
learn.show_results()

In [ ]:
learn.show_probas()
#was sagt probability aus? accuracy vs probability?

In [ ]:
learn.plot_confusion_matrix()

In [ ]:
learn.plot_top_losses(X[splits[1]], y[splits[1]], largest=True)